In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
# !pip3 install tensorflow

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle
from utils_misc import pickle_load_or_create

2024-06-25 16:47:04.867917: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 16:47:05.336054: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 16:47:06.362641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
model = pickle_load_or_create('../models/frames_skillborder_convLSTM_model_history', lambda: None, True)
print(model)

{'accuracy': [0.3533805012702942, 0.3549528419971466],
 'loss': [10.40020751953125, 10.396940231323242],
 'val_accuracy': [0.47440946102142334, 0.47440946102142334],
 'val_loss': [8.471517562866211, 8.471517562866211]}

In [9]:
from DataGeneratorBordersDB import DataGeneratorSkillBorders


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [10]:
config = pickle_load_or_create('convLSTM_192_3x3_rgb', lambda:{
    'convolution': (3,3),
    'time_length':8,
    'dim':128,
    'rgb':True,
    'unique_labels': 4,
}, config=True)
config

{'convolution': (3, 3),
 'time_length': 8,
 'dim': 128,
 'rgb': True,
 'unique_labels': 4}

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, TimeDistributed, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

if model is None:
    model = Sequential()    
    model.add(ConvLSTM2D(filters=64, kernel_size=config['convolution'],
                         input_shape=(config['time_length'], config['dim'],  config['dim'], 3 if config['rgb'] else 1),
                         padding='same', return_sequences=True))
    model.add(BatchNormalization())
    
    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                         padding='same', return_sequences=True))
    model.add(BatchNormalization())
    
    # Uncomment if you want to add more ConvLSTM2D layers
    # model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
    #                      padding='same', return_sequences=True))
    # model.add(BatchNormalization())
    
    # Apply TimeDistributed Dense layer to each time step
    model.add(TimeDistributed(Flatten()))  # Flatten each frame
    model.add(TimeDistributed(Dense(128, activation='relu')))
    model.add(TimeDistributed(Dense(config['unique_labels'], activation='softmax')))
    
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
else:
    model = model.model

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 8, 128, 128,    │       154,624 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 8, 128, 128,    │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 8, 128, 128,    │       295,168 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 8, 128, 128,    │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 8, 1048576)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 8, 128)         │   134,217,856 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 8, 4)           │           516 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 404,005,518 (1.51 GB)

 Trainable params: 134,668,420 (513.72 MB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 269,336,842 (1.00 GB)

In [16]:
# Parameters
params = {'dim': (config['dim'],config['dim']),
          'time_length': config['time_length'],
          'n_classes': config['unique_labels'],
          'n_channels': 3 if config['rgb'] else 1,
          'shuffle': True,
}

training_generator = DataGeneratorSkillBorders(train=True, **params)
test_generator = DataGeneratorSkillBorders(train=False, **params)

connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
   videoID  batches_in_video  random_order  total_batches
0        3               186             1            186
1        6               112             2            298
2        2               180             3            478
3        4               260             4            738
4        7               168             5            906
5        5               318             6           1224
6        1                92             7           1316
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
   videoID  batches_in_video  random_order  total_batches
0        8               127             1            127
1        9                87             2            214


In [17]:
training_generator.batch_order

,videoID,batches_in_video,random_order,total_batches
0,3,186,1,186
1,6,112,2,298
2,2,180,3,478
3,4,260,4,738
4,7,168,5,906
5,5,318,6,1224
6,1,92,7,1316


In [18]:
%%time
X, y = training_generator.__getitem__(25)

 getitem, batch_nr 25 train is: True
CPU times: user 904 ms, sys: 43.4 ms, total: 947 ms
Wall time: 325 ms


In [19]:
X.shape

(1, 8, 128, 128, 3)

In [20]:
X

array([[[[[0.3254902 , 0.3372549 , 0.3372549 ],
          [0.32156863, 0.33333333, 0.33333333],
          [0.3254902 , 0.3372549 , 0.3372549 ],
          ...,
          [0.28235294, 0.29411765, 0.29411765],
          [0.28627451, 0.29803922, 0.29803922],
          [0.29411765, 0.30588235, 0.30588235]],

         [[0.32941176, 0.34117647, 0.34117647],
          [0.31372549, 0.3254902 , 0.3254902 ],
          [0.31372549, 0.3254902 , 0.3254902 ],
          ...,
          [0.29411765, 0.30588235, 0.30588235],
          [0.29411765, 0.30588235, 0.30588235],
          [0.29411765, 0.30588235, 0.30588235]],

         [[0.33333333, 0.34509804, 0.34509804],
          [0.32941176, 0.34117647, 0.34117647],
          [0.31764706, 0.32941176, 0.32941176],
          ...,
          [0.29411765, 0.30588235, 0.30588235],
          [0.30980392, 0.30980392, 0.30980392],
          [0.29803922, 0.30980392, 0.30980392]],

         ...,

         [[0.74901961, 0.71764706, 0.67058824],
          [0.71372549,

In [21]:
y.shape

(1, 8, 1)

In [22]:
y

array([[[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [3],
        [3]]])

In [23]:
y.shape


(1, 8, 1)

In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=2, 
                    validation_data=test_generator, shuffle=False)

len called len is: 1316
len called len is: 1316
 getitem, batch_nr 0 train is: True
 getitem, batch_nr 1 train is: True
Epoch 1/2
len called len is: 1316
len called len is: 1316
 getitem, batch_nr 0 train is: True
 getitem, batch_nr 1 train is: True
   1/1316 ━━━━━━━━━━━━━━━━━━━━ 3:38:59 10s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 2 train is: True
   2/1316 ━━━━━━━━━━━━━━━━━━━━ 1:44:07 5s/step - accuracy: 1.0000 - loss: 3.5763e-07  getitem, batch_nr 3 train is: True
   3/1316 ━━━━━━━━━━━━━━━━━━━━ 1:44:50 5s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 4 train is: True
   4/1316 ━━━━━━━━━━━━━━━━━━━━ 1:44:54 5s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 5 train is: True
   5/1316 ━━━━━━━━━━━━━━━━━━━━ 1:46:19 5s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 6 train is: True
   6/1316 ━━━━━━━━━━━━━━━━━━━━ 1:48:16 5s/step - accuracy: 1.0000 - loss: 3.5763e-07 getitem, batch_nr 7 train is: True
   7/1316 ━━━━━━━━━━━━━━━━━━

In [25]:
pd.DataFrame(history.history)

,accuracy,loss,val_accuracy,val_loss
0,0.440445,9.018946,0.428738,9.199925
1,0.409195,9.522095,0.409463,9.518359


In [26]:
model.fit?

Signature:
model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)
Docstring:
Trains the model for a fixed number of epochs (dataset iterations).

Args:
    x: Input data. It could be:
        - A NumPy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
        - A tensor, or a list of tensors
        (in case the model has multiple inputs).
        - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
        - A `tf.data.Dataset`. Should return a tuple
        of either `(inputs, targets)` or
        `(inputs, targets, sample_weights)`.
        - A `keras.utils.PyDataset` returning `(inputs,
      

In [27]:
with open('../models/frames_skillborder_convLSTM_model_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)